In [2]:
from dotenv import load_dotenv
import os 

load_dotenv(".env", override=True)
# print(os.environ.get("NEO4J_URI"))

True

In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

api_key = os.environ.get("OPENAI_API_KEY")

critic_llm = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
generator_llm = ChatOllama(model="llama3")
langchain_embedding = OpenAIEmbeddings(api_key=api_key)
# langchain_embedding = OllamaEmbeddings(model="ollama3")

In [8]:
critic_llm.invoke("Tell me a medium sized story")

AIMessage(content="Once upon a time, in a small village nestled in the rolling hills of Provence, France, there lived a young girl named Sophie. She was a curious and adventurous child, with a mop of curly brown hair and a smile that could light up the entire village.\n\nSophie loved to explore the surrounding countryside, where she would spend hours chasing after butterflies and picking wildflowers for her mother's famous bouquets. But Sophie had a special talent - she was a master baker. She spent every spare moment in her family's small bakery, learning the secrets of traditional French bread-making from her father, Pierre.\n\nAs the summer solstice approached, Sophie's village was preparing for its annual Fête de la Saint-Pierre. The entire community would come together to celebrate the patron saint of fishermen, with music, dance, and of course, an abundance of delicious food.\n\nSophie knew that she had to create something truly special for the fête. She spent hours experimenting

In [ ]:
from ragas.metrics import context_precision, faithfulness, answer_relevancy, context_recall

In [10]:
from langchain_community.graphs import Neo4jGraph


aura_host = os.environ.get('NEO4J_AURA_URL')
aura_user = os.environ.get('NEO4J_AURA_USER')
aura_password = os.environ.get('NEO4J_AURA_PASS')


graph_db = Neo4jGraph(url=aura_host, username=aura_user, password=aura_password)

In [12]:
from langchain_core.documents import Document

langchain_data = {}
documents = []

result = graph_db.query('''MATCH (subject)-[predicate]->(object) WHERE NOT (subject:Document)
RETURN toString(subject.id) + " " + type(predicate) + " " + toString(object.id) AS triple''')
n_results = len(result)
for record in result:
    # print(record)
    str_record = str(record["triple"])
    print(str_record)
    doc = Document(str_record)
    documents.append(doc)
    
graph_db._driver.close()

Santa Clara (Portugal) OWES_MONEY Atlético- MG
Vasco OWES_MONEY Atlético- MG
Flamengo OWES_MONEY Atlético- MG
Ceará OWES_MONEY Atlético- MG
Fluminense OWES_MONEY Atlético- MG
Cruzeiro INTERESTED_IN_SIGNING Gabriel Grando
Gabriel Grando PLAYS_FOR Cruzeiro
Gabriel Grando LOANED_FROM Grêmio
Gabriel Grando HAS_POSTINGS musics and love songs
Gabriel Grando NEEDS_CONFIDENCE Cruzeiro
MLS (Estados Unidos) OWES_MONEY Atlético- MG
Santos HAS_PLAYER Fábio Carille
Santos HAS_PLAYER Vinicius Balieiro
Santos HAS_PLAYER Bruno Marques
Santos HAS_PLAYER Dodô
Santos OWES_MONEY Atlético- MG
Santos WANTS_TO_SIGN Jair e Pedrinho
Santos MAY_SIGN João Basso, Nathan, Lucas Pires and Carabajal
Fortaleza PLAYED_AGAINST Vasco
Fortaleza OWES_MONEY Atlético- MG
Juventude OWES_MONEY Atlético- MG
Pumas (México) OWES_MONEY Atlético- MG
Bahia OWES_MONEY Atlético- MG
São Paulo WORKS_WITH Welington
São Paulo OWES_MONEY Atlético- MG
São Paulo MAY_LOSE Welington
Allan TRANSFERRED_TO Flamengo
Metalist (Ucrânia) OWES_MONEY 

In [13]:
from ragas.testset import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    langchain_embedding
)

distribution = {simple: 1}
question_types = len(distribution.keys())
testset = generator.generate_with_langchain_docs(documents, test_size=question_types*n_results, distributions={simple: 1})


c:\Users\felip\anaconda3\envs\local-ragas\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Adding documents
Adding nodes
Waiting for executor to finish
Creating executor job=====================================


embedding nodes:   0%|          | 0/386 [00:00<?, ?it/s]

In [ ]:

df = testset.to_pandas()
# df.to_csv('testset.csv')
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"Based on the given context, I would form a que...",[Botafogo PLAYS_AGAINST Atlético-Go],Botafogo is playing against Atlético-Go,simple,[{}],True
1,"Based on the given context, I would form a que...",[Igor Vinicius RECEIVED_YELLOW_CARD_FROM São P...,Receiving a yellow card in football means that...,simple,[{}],True


In [ ]:
df[df["ground_truth"] != "nan"]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"Based on the given context, I would form a que...",[Botafogo PLAYS_AGAINST Atlético-Go],Botafogo is playing against Atlético-Go,simple,[{}],True
1,"Based on the given context, I would form a que...",[Igor Vinicius RECEIVED_YELLOW_CARD_FROM São P...,Receiving a yellow card in football means that...,simple,[{}],True


In [ ]:
df.iloc[0]["question"]

'Based on the given context, I would form a question that can be fully answered using the topic "Atlético-Go". Here\'s my attempt:\n\ncontext: "Botafogo PLAYS_AGAINST Atlético-Go"\nkeyphrase: "Atlético-Go"\nquestion: "Who is Botafogo playing against in this match?"\n\nLet me know if I got it right!'